## Import and Load datasets

In [1]:
import pandas as pd
import numpy as np

In [2]:
df2017 = pd.read_csv('/content/FY2017-4050-County-Level_Data.csv', encoding= 'unicode_escape', )
df2018 = pd.read_csv('/content/FY18_4050_FMRs_rev.csv', encoding= 'unicode_escape')
df2019 = pd.read_csv('/content/FY2019_4050_FMRs_rev2.csv', encoding= 'unicode_escape')
df2020 = pd.read_csv('/content/FY20_4050_FMRs_rev.csv', encoding= 'unicode_escape')
df2021 = pd.read_csv('/content/FY21_4050_FMRs_rev.csv', encoding= 'unicode_escape')

## Clean datasets

In [11]:
def wrangle0(x0):
  x0 = x0.copy()
  x0 = x0.loc[x0['metro'] == 1]
  x0 = x0[['fips2010', 'metro_code', 'areaname', 'state_alpha','fmr_0']]
  x0 = x0.rename(columns = {'fmr_0': 2021})
  return x0

In [4]:
df2017_frm0 = wrangle0(df2017)

In [6]:
df2018_frm0 = wrangle0(df2018)

In [8]:
df2019_frm0 = wrangle0(df2019)

In [10]:
df2020_frm0 = wrangle0(df2020)

In [12]:
df2021_frm0 = wrangle0(df2021)

In [49]:
def wrangle1(x1):
  x1 = x1.copy()
  x1 = x1.loc[x1['metro'] == 1]
  x1 = x1[['fips2010', 'metro_code', 'areaname', 'state_alpha','fmr_1']]
  x1 = x1.rename(columns = {'fmr_1': 2021})
  return x1

In [37]:
df2017_fmr1 = wrangle1(df2017)

In [40]:
df2018_frm1 = wrangle1(df2018)

In [44]:
df2019_frm1 = wrangle1(df2019)

In [47]:
df2020_frm1 = wrangle1(df2020)

In [50]:
df2021_frm1 = wrangle1(df2021)

## Merge fmr0s and fmr1s

In [53]:
df_fmr0 = df2017_frm0.merge(df2018_frm0, on=['fips2010', 'metro_code', 'areaname', 'state_alpha'])
df_fmr0 = df_fmr0.merge(df2019_frm0, on=['fips2010', 'metro_code', 'areaname', 'state_alpha'])
df_fmr0 = df_fmr0.merge(df2020_frm0, on=['fips2010', 'metro_code', 'areaname', 'state_alpha'])
df_fmr0 = df_fmr0.merge(df2021_frm0, on=['fips2010', 'metro_code', 'areaname', 'state_alpha'])

In [60]:
print(df_fmr0.shape)
df_fmr0.head()

(1963, 9)


,fips2010,metro_code,areaname,state_alpha,2017,2018,2019,2020,2021
0,100199999,METRO33860M33860,"Montgomery, AL MSA",AL,587,588,581,583,640
1,100399999,METRO19300M19300,"Daphne-Fairhope-Foley, AL MSA",AL,807,729,756,744,718
2,100799999,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",AL,665,692,706,804,817
3,100999999,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",AL,665,692,706,804,817
4,101599999,METRO11500M11500,"Anniston-Oxford-Jacksonville, AL MSA",AL,437,477,457,468,482


In [57]:
df_fmr1 = df2017_fmr1.merge(df2018_frm1, on=['fips2010', 'metro_code', 'areaname', 'state_alpha'])
df_fmr1 = df_fmr1.merge(df2019_frm1, on=['fips2010', 'metro_code', 'areaname', 'state_alpha'])
df_fmr1 = df_fmr1.merge(df2020_frm1, on=['fips2010', 'metro_code', 'areaname', 'state_alpha'])
df_fmr1 = df_fmr1.merge(df2021_frm1, on=['fips2010', 'metro_code', 'areaname', 'state_alpha'])

In [61]:
print(df_fmr1.shape)
df_fmr1.head()

(1963, 9)


,fips2010,metro_code,areaname,state_alpha,2017,2018,2019,2020,2021
0,100199999,METRO33860M33860,"Montgomery, AL MSA",AL,682,691,696,702,766
1,100399999,METRO19300M19300,"Daphne-Fairhope-Foley, AL MSA",AL,847,762,773,749,723
2,100799999,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",AL,751,768,763,861,871
3,100999999,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",AL,751,768,763,861,871
4,101599999,METRO11500M11500,"Anniston-Oxford-Jacksonville, AL MSA",AL,474,511,513,531,549


## Clean Merged DataFrames
- split area name to remove 'MSA', 'HUD Metro FMR Area'
- explode cities to separate hyphenated cities

In [62]:
def explode_str(df, col='areaname', sep='-'):
    s = df[col]
    i = np.arange(len(s)).repeat(s.str.count(sep) +1)
    return df.iloc[i].assign(**{col: sep.join(s).split(sep)})

In [74]:
def new_wrangle(x):
  x = x.copy()
  x['areaname'] = x['areaname'].str.split(',', 1, expand=True)[0]
  x['areaname'] = x['areaname'].str.strip()
  x = explode_str(x)
  x = x.drop(columns = ['fips2010'])

  return x

### FMR0

In [76]:
df_fmr0 = new_wrangle(df_fmr0)

In [78]:
df_fmr0['areaname'] = df_fmr0['areaname'].str.strip()
df_fmr0['state_alpha'] = df_fmr0['state_alpha'].str.strip()
df_fmr0['city, state'] = df_fmr0['areaname'] + ', ' + df_fmr0['state_alpha']

In [79]:
print(df_fmr0.shape)
df_fmr0.head()

(3228, 9)


,metro_code,areaname,state_alpha,2017,2018,2019,2020,2021,"city, state"
0,METRO33860M33860,Montgomery,AL,587,588,581,583,640,"Montgomery, AL"
1,METRO19300M19300,Daphne,AL,807,729,756,744,718,"Daphne, AL"
1,METRO19300M19300,Fairhope,AL,807,729,756,744,718,"Fairhope, AL"
1,METRO19300M19300,Foley,AL,807,729,756,744,718,"Foley, AL"
2,METRO13820M13820,Birmingham,AL,665,692,706,804,817,"Birmingham, AL"


In [80]:
df_fmr0.to_csv('fmr0.csv', index=False)

### FMR1

In [81]:
df_fmr1 = new_wrangle(df_fmr1)

In [82]:
df_fmr1['areaname'] = df_fmr1['areaname'].str.strip()
df_fmr1['state_alpha'] = df_fmr1['state_alpha'].str.strip()
df_fmr1['city, state'] = df_fmr1['areaname'] + ', ' + df_fmr1['state_alpha']

In [83]:
print(df_fmr1.shape)
df_fmr1.head()

(3228, 9)


,metro_code,areaname,state_alpha,2017,2018,2019,2020,2021,"city, state"
0,METRO33860M33860,Montgomery,AL,682,691,696,702,766,"Montgomery, AL"
1,METRO19300M19300,Daphne,AL,847,762,773,749,723,"Daphne, AL"
1,METRO19300M19300,Fairhope,AL,847,762,773,749,723,"Fairhope, AL"
1,METRO19300M19300,Foley,AL,847,762,773,749,723,"Foley, AL"
2,METRO13820M13820,Birmingham,AL,751,768,763,861,871,"Birmingham, AL"


In [84]:
df_fmr1.to_csv('fmr1.csv', index=False)

## Forecasting

In [86]:
!pip install prophet

     |████████████████████████████████| 71kB 4.4MB/s 
     |████████████████████████████████| 51kB 3.3MB/s 
     |████████████████████████████████| 184kB 14.3MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-cp37-none-any.whl size=6638712 sha256=cd178320714b76c0d5caebeb41b6f39e971bf37a7ed602b52fb3bc07ccd4d4d3
  Stored in directory: /root/.cache/pip/wheels/92/1d/02/bc0a91bd3a84e060722075d2b0b68e875e37ec7dc49451d513
Successfully built prophet
ERROR: fbprophet 0.7.1 has requirement cmdstanpy==0.9.5, but you'll have cmdstanpy 0.9.68 which is incompatible.
  Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5


In [87]:
from prophet import Prophet

In [85]:
fmr0 = pd.read_csv('/content/fmr0.csv')

### Setting fmr0 
- select specific rows
- melt dataframe and get columns necessary to run prophet

In [88]:
print(fmr0.shape)
fmr0.head()

(3228, 9)


,metro_code,areaname,state_alpha,2017,2018,2019,2020,2021,"city, state"
0,METRO33860M33860,Montgomery,AL,587,588,581,583,640,"Montgomery, AL"
1,METRO19300M19300,Daphne,AL,807,729,756,744,718,"Daphne, AL"
2,METRO19300M19300,Fairhope,AL,807,729,756,744,718,"Fairhope, AL"
3,METRO19300M19300,Foley,AL,807,729,756,744,718,"Foley, AL"
4,METRO13820M13820,Birmingham,AL,665,692,706,804,817,"Birmingham, AL"


In [95]:
fmr0_melt = fmr0.copy()

In [96]:
fmr0_melt = fmr0_melt.drop(columns=['metro_code', 'areaname', 'state_alpha'])

In [98]:
# to get in the form prophet needs
fmr0_melt = (fmr0_melt.melt(id_vars=['city, state'], 
                         var_name='ds', 
                         value_name='y')
                         ).reset_index(drop=True)

In [99]:
fmr0_melt.head()

,"city, state",ds,y
0,"Montgomery, AL",2017,587
1,"Daphne, AL",2017,807
2,"Fairhope, AL",2017,807
3,"Foley, AL",2017,807
4,"Birmingham, AL",2017,665


In [101]:
# create city list
cities_list = list(fmr0_melt['city, state'].unique())

In [102]:
len(cities_list)

910

In [103]:
def rnd_series(city):
    subset = fmr0_melt[fmr0_melt['city, state']==city]
    dates =(pd.DataFrame({'ds':pd.to_datetime(fmr0_melt['ds'])}))
    return subset

In [104]:
series = [rnd_series(city) for city in cities_list]

In [105]:
series[0]

,"city, state",ds,y
0,"Montgomery, AL",2017,587
14,"Montgomery, AL",2017,587
28,"Montgomery, AL",2017,587
31,"Montgomery, AL",2017,587
3228,"Montgomery, AL",2018,588
3242,"Montgomery, AL",2018,588
3256,"Montgomery, AL",2018,588
3259,"Montgomery, AL",2018,588
6456,"Montgomery, AL",2019,581
6470,"Montgomery, AL",2019,581


### Running Prophet on df

In [106]:
def run_prophet(series):
    model = Prophet()
    model.fit(series)
    forecast = model.make_future_dataframe(periods=10, freq='Y')
    forecast = model.predict(forecast)
    forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    forecast['city, state'] = series['city, state'].iloc[0]
    forecast = forecast[['city, state','ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].astype(float)
    forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].apply(np.ceil)
    return forecast

In [107]:
f = run_prophet(series[0])
f.head()

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.


,"city, state",ds,yhat,yhat_lower,yhat_upper
0,"Montgomery, AL",2017-01-01,588.0,587.0,588.0
1,"Montgomery, AL",2018-01-01,589.0,588.0,589.0
2,"Montgomery, AL",2019-01-01,581.0,581.0,582.0
3,"Montgomery, AL",2020-01-01,583.0,583.0,584.0
4,"Montgomery, AL",2021-01-01,640.0,640.0,641.0


In [108]:
for i in range(len(series)):
    f = run_prophet(series[i])
    f.to_csv('fmr0_predictions.csv', mode='a', index='False')

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seas

In [109]:
rental_prediction = pd.read_csv('fmr0_predictions.csv')

In [110]:
rental_prediction = rental_prediction.loc[rental_prediction['city, state'] != 'city, state']

In [111]:
rental_prediction.to_csv('fmr0_predictions.csv', index=False)

In [112]:
rent_long = rental_prediction[['city, state', 'ds', 'yhat']]

In [113]:
rent_long.dtypes

city, state    object
ds             object
yhat           object
dtype: object

In [114]:
rent_long['yhat'] = pd.to_numeric(rent_long['yhat'],errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [115]:
rent_long.head()

,"city, state",ds,yhat
0,"Montgomery, AL",2017-01-01,588.0
1,"Montgomery, AL",2018-01-01,589.0
2,"Montgomery, AL",2019-01-01,581.0
3,"Montgomery, AL",2020-01-01,583.0
4,"Montgomery, AL",2021-01-01,640.0


In [116]:
rent_long = rent_long.pivot_table(index = 'city, state',
                                  columns = 'ds',
                                  values = 'yhat')

In [141]:
rent_long = rent_long.iloc[3:]

In [142]:
rent_long

ds,2017-01-01,2018-01-01,2019-01-01,2020-01-01,2021-01-01,2021-12-31,2022-12-31,2023-12-31,2024-12-31,2025-12-31,2026-12-31,2027-12-31,2028-12-31,2029-12-31,2030-12-31
"city, state",,,,,,,,,,,,,,,
"Abilene, TX",566.0,566.0,598.0,632.0,708.0,679.0,694.0,702.0,851.0,872.0,886.0,894.0,1043.0,1064.0,1079.0
"Acadia Parish, LA",419.0,455.0,491.0,527.0,546.0,600.0,637.0,674.0,691.0,728.0,765.0,802.0,819.0,856.0,892.0
"Aguadilla, PR",357.0,342.0,365.0,363.0,381.0,346.0,337.0,326.0,356.0,348.0,338.0,328.0,358.0,349.0,340.0
"Aibonito, PR",346.0,312.0,334.0,327.0,337.0,284.0,261.0,236.0,262.0,240.0,216.0,191.0,217.0,195.0,172.0
"Akron, OH",516.0,538.0,527.0,557.0,566.0,617.0,647.0,677.0,687.0,717.0,747.0,777.0,788.0,817.0,847.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"York, ME",822.0,876.0,918.0,948.0,1067.0,968.0,960.0,939.0,1209.0,1213.0,1205.0,1184.0,1453.0,1457.0,1449.0
"York, PA",575.0,601.0,620.0,632.0,640.0,634.0,623.0,606.0,701.0,698.0,688.0,670.0,766.0,763.0,752.0
"Youngstown, OH",480.0,485.0,474.0,483.0,512.0,497.0,503.0,507.0,549.0,556.0,561.0,566.0,608.0,615.0,620.0


In [143]:
rent_long.to_csv('fmr0_long.csv', index=False )

## Forecasting 2

In [144]:
fmr1 = pd.read_csv('/content/fmr1.csv')

### Setting fmr1 
- select specific rows
- melt dataframe and get columns necessary to run prophet

In [145]:
print(fmr1.shape)
fmr1.head()

(3228, 9)


,metro_code,areaname,state_alpha,2017,2018,2019,2020,2021,"city, state"
0,METRO33860M33860,Montgomery,AL,682,691,696,702,766,"Montgomery, AL"
1,METRO19300M19300,Daphne,AL,847,762,773,749,723,"Daphne, AL"
2,METRO19300M19300,Fairhope,AL,847,762,773,749,723,"Fairhope, AL"
3,METRO19300M19300,Foley,AL,847,762,773,749,723,"Foley, AL"
4,METRO13820M13820,Birmingham,AL,751,768,763,861,871,"Birmingham, AL"


In [146]:
fmr1_melt = fmr1.copy()

In [147]:
fmr1_melt = fmr1_melt.drop(columns=['metro_code', 'areaname', 'state_alpha'])

In [148]:
# to get in the form prophet needs
fmr1_melt = (fmr1_melt.melt(id_vars=['city, state'], 
                         var_name='ds', 
                         value_name='y')
                         ).reset_index(drop=True)

In [149]:
fmr1_melt.head()

,"city, state",ds,y
0,"Montgomery, AL",2017,682
1,"Daphne, AL",2017,847
2,"Fairhope, AL",2017,847
3,"Foley, AL",2017,847
4,"Birmingham, AL",2017,751


In [150]:
# create city list
cities_list = list(fmr1_melt['city, state'].unique())

In [151]:
len(cities_list)

910

In [152]:
def rnd_series(city):
    subset = fmr1_melt[fmr1_melt['city, state']==city]
    dates =(pd.DataFrame({'ds':pd.to_datetime(fmr1_melt['ds'])}))
    return subset

In [153]:
series = [rnd_series(city) for city in cities_list]

In [154]:
series[0]

,"city, state",ds,y
0,"Montgomery, AL",2017,682
14,"Montgomery, AL",2017,682
28,"Montgomery, AL",2017,682
31,"Montgomery, AL",2017,682
3228,"Montgomery, AL",2018,691
3242,"Montgomery, AL",2018,691
3256,"Montgomery, AL",2018,691
3259,"Montgomery, AL",2018,691
6456,"Montgomery, AL",2019,696
6470,"Montgomery, AL",2019,696


### Running Prophet on df

In [155]:
def run_prophet(series):
    model = Prophet()
    model.fit(series)
    forecast = model.make_future_dataframe(periods=10, freq='Y')
    forecast = model.predict(forecast)
    forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    forecast['city, state'] = series['city, state'].iloc[0]
    forecast = forecast[['city, state','ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].astype(float)
    forecast[['yhat', 'yhat_lower', 'yhat_upper']] = forecast[['yhat', 'yhat_lower', 'yhat_upper']].apply(np.ceil)
    return forecast

In [156]:
f = run_prophet(series[0])
f.head()

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.


,"city, state",ds,yhat,yhat_lower,yhat_upper
0,"Montgomery, AL",2017-01-01,683.0,682.0,683.0
1,"Montgomery, AL",2018-01-01,691.0,691.0,692.0
2,"Montgomery, AL",2019-01-01,697.0,696.0,697.0
3,"Montgomery, AL",2020-01-01,703.0,702.0,703.0
4,"Montgomery, AL",2021-01-01,766.0,766.0,767.0


In [157]:
for i in range(len(series)):
    f = run_prophet(series[i])
    f.to_csv('fmr1_predictions.csv', mode='a', index='False')

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 15.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seas

In [158]:
rental_prediction1 = pd.read_csv('fmr1_predictions.csv')

In [159]:
rental_prediction1 = rental_prediction1.loc[rental_prediction1['city, state'] != 'city, state']

In [160]:
rental_prediction1.to_csv('fmr1_predictions.csv', index=False)

In [161]:
rent_long1 = rental_prediction1[['city, state', 'ds', 'yhat']]

In [162]:
rent_long1.dtypes

city, state    object
ds             object
yhat           object
dtype: object

In [163]:
rent_long1['yhat'] = pd.to_numeric(rent_long1['yhat'],errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [164]:
rent_long1.head()

,"city, state",ds,yhat
0,"Montgomery, AL",2017-01-01,683.0
1,"Montgomery, AL",2018-01-01,691.0
2,"Montgomery, AL",2019-01-01,697.0
3,"Montgomery, AL",2020-01-01,703.0
4,"Montgomery, AL",2021-01-01,766.0


In [165]:
rent_long1 = rent_long1.pivot_table(index = 'city, state',
                                  columns = 'ds',
                                  values = 'yhat')

In [166]:
rent_long1 = rent_long1.iloc[3:]

In [167]:
rent_long1

ds,2017-01-01,2018-01-01,2019-01-01,2020-01-01,2021-01-01,2021-12-31,2022-12-31,2023-12-31,2024-12-31,2025-12-31,2026-12-31,2027-12-31,2028-12-31,2029-12-31,2030-12-31
"city, state",,,,,,,,,,,,,,,
"Abilene, TX",617.0,606.0,621.0,656.0,734.0,801.0,913.0,1050.0,818.0,904.0,1015.0,1152.0,920.0,1006.0,1118.0
"Acadia Parish, LA",463.0,487.0,510.0,532.0,551.0,574.0,594.0,614.0,641.0,662.0,682.0,701.0,728.0,749.0,769.0
"Aguadilla, PR",375.0,353.0,373.0,372.0,393.0,371.0,371.0,371.0,387.0,387.0,386.0,386.0,403.0,402.0,402.0
"Aibonito, PR",362.0,327.0,364.0,359.0,370.0,361.0,361.0,362.0,377.0,377.0,378.0,378.0,393.0,393.0,394.0
"Akron, OH",599.0,617.0,636.0,655.0,665.0,695.0,715.0,736.0,741.0,761.0,782.0,802.0,807.0,827.0,848.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"York, ME",894.0,945.0,1040.0,1009.0,1119.0,940.0,905.0,870.0,989.0,955.0,920.0,885.0,1004.0,970.0,935.0
"York, PA",691.0,717.0,732.0,737.0,737.0,718.0,693.0,658.0,778.0,763.0,738.0,703.0,823.0,808.0,783.0
"Youngstown, OH",547.0,556.0,545.0,558.0,585.0,576.0,582.0,585.0,642.0,650.0,656.0,659.0,717.0,725.0,730.0


In [168]:
rent_long1.to_csv('fmr1_long.csv', index=False )